## Auto-Summarization of Congressional Bills

Acknowledgments: This code builds on Seq2Seq modeling availabile here and HW4 of Advanced Machine Learning written by Zewei Chu.

Abstract:

In this paper, we describe a Sequence to Sequence neural network with multilayered Long Short-Term Memory (LSTM) networks. We use a novel data set of Congressional bills and human-generated summaries to train the model. We find that WHAT:

Introduction:

The effectiveness of abstract summarization techniques is very limited on complex and lengthy texts. 

In [1]:
%conda install torchtext -y -c pytorch

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::nbclassic==0.2.6=pyhd8ed1ab_0
  - conda-forge/noarch::smart_open==2.2.1=pyh9f0ad1d_0
  - conda-forge/linux-64::blaze==0.11.3=py36_0
  - defaults/linux-64::_anaconda_depends==5.1.0=py36_2
  - conda-forge/noarch::requests==2.25.1=pyhd3deb0d_0
  - conda-forge/noarch::jupyterlab==3.0.9=pyhd8ed1ab_0
  - conda-forge/noarch::python-language-server==0.36.2=pyhd8ed1ab_0
  - conda-forge/noarch::jupyterlab_server==2.3.0=pyhd8ed1ab_0
  - conda-forge/noarch::google-api-core==1.25.1=pyh44b312d_0
  - conda-forge/noarch::pyls-black==0.4.6=pyh9f0ad1d_0
  - fastai/noarch::fastai==1.0.61=1
  - conda-forge/noarch::pathy==0.4.0=pyhd8ed1ab_0
  - conda-forge/noarch::black==20.8b1=py_1
  - conda-forge/linux-64::spacy==3.0.3=py36h355b2fd_0
  - conda-forge/linux-64::anyio==2.1.0=py36h5fab9bb_0
  - conda-forge/linux-64::jupyter_server==1.4

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import GloVe
import numpy as np
import random
import os
import pandas as pd
from numpy import floor
from numpy.random import shuffle
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import Vocab
import torch.nn.functional as F
from collections import Counter
from functools import partial
from ast import literal_eval
import boto3, os
import sagemaker
import matplotlib.pyplot as plt
%matplotlib inline

In [46]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED = 1

print(DEVICE)

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if USE_CUDA:
    torch.cuda.manual_seed(SEED)

COLAB = False
AWS = True
DEVELOPING = True
SINGLE_RECORD = False

if DEVELOPING:
    if SINGLE_RECORD:
        print('Training on single record')
        BATCH_SIZE = 1
        EMBEDDING_SIZE = 300
        TRAINING_SIZE = 1
        VALIDATION_SIZE = 0
        TESTING_SIZE = 0
        MAX_SUMMARY_LENGTH = 5
        MAX_BILL_LENGTH = 8
    else:
        print('Small development version')
        BATCH_SIZE = 20
        EMBEDDING_SIZE = 300
        DATA_FILE = 'Minimum_Length_Sample.csv'
        TRAINING_SIZE = .7
        VALIDATION_SIZE = .2
        TESTING_SIZE = .1
else:
    print('Full version')
    BATCH_SIZE = 4
    EMBEDDING_SIZE = 300
    DATA_FILE = 'Cleaned_Summaries_And_Bills.csv'
    TRAINING_SIZE = .7
    VALIDATION_SIZE = .2
    TESTING_SIZE = .1


cuda
Small development version


In [4]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    PATH = '/content/drive/MyDrive/'

elif AWS:
    sagemaker_session = sagemaker.Session()
    role = sagemaker.get_execution_role()
    bucket = 'capp-20235'
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket=bucket, Key=DATA_FILE)
else:
    PATH = './'

if SINGLE_RECORD:
    DATA_SET = pd.DataFrame({'bill_clean':[['<sos>', 'make', 'all', 'drugs', 'legal','for','citizens', '<eos>']], 'summary_clean':[['<sos>', 'make', 'drugs', 'legal', '<eos>']]})
elif AWS:
    DATA_SET = pd.read_csv(response['Body'], converters={'summary_clean': literal_eval, 'bill_clean': literal_eval})
else:
    DATA_SET = pd.read_csv(PATH + DATA_FILE, converters={'summary_clean': literal_eval, 'bill_clean': literal_eval})

print(DATA_SET.head(2))

            id                                            summary  \
0  id113hr3079    Provides for the relief of Jesus Garcia Flores.   
1  id113hr3169  IRS Rulemaking Fairness Act of 2013 - Expands ...   

                    name                                               link  \
0  BILLS-113hr3079ih.xml  https://www.govinfo.gov/bulkdata/BILLS/113/1/h...   
1  BILLS-113hr3169ih.xml  https://www.govinfo.gov/bulkdata/BILLS/113/1/h...   

                                                text  \
0  <?xml version="1.0"?>\n<?xml-stylesheet type="...   
1  <?xml version="1.0"?>\n<?xml-stylesheet type="...   

                                       summary_clean  \
0  [<sos>, provides, for, the, relief, of, jesus,...   
1  [<sos>, irs, rulemaking, fairness, act, of, ##...   

                                          bill_clean  
0  [<sos>, :, for, the, relief, of, jesus, garcia...  
1  [<sos>, :, irs, rulemaking, fairness, act, of,...  


## Preprocessing of the data

In [5]:
def trim_dataset(df, bottom_k_pct, top_k_pct):
    '''
    Remove the top and bottom n% records from the bills and summaries.
    Expects tokenized and cleaned dataset.
    Pass in pct as decimals.
    '''
    df['summary_length'] = df.summary_clean.apply(lambda x: len(x))
    df['bill_length'] = df.bill_clean.apply(lambda x: len(x))
    df['summary_rank'] = df.summary_length.rank(pct=True)
    df['bill_rank'] = df.bill_length.rank(pct=True)
    cut_df = df[(df.summary_rank >= bottom_k_pct) & (df.summary_rank <= top_k_pct) & (
        df.bill_rank >= bottom_k_pct) & (df.bill_rank <= top_k_pct) & (df.summary_length <= df.bill_length)]

    max_summary = cut_df.summary_length.max()
    max_bill = cut_df.bill_length.max()

    print('Cut ' + str(df.shape[0] - cut_df.shape[0]) + ' records.')
    print('Count of records remaining: ', cut_df.shape[0])
    print(f'New min summary length is {cut_df.summary_length.min()}')
    print(f'New max summary length is {cut_df.summary_length.max()}')
    print(f'New min bill length is {cut_df.bill_length.min()}')
    print(f'New max bill length is {cut_df.bill_length.max()}')
    print(f'Compression of summaries to bills is {compression(cut_df)}')
    del cut_df['bill_length']
    del cut_df['summary_length']
    del cut_df['summary_rank']
    del cut_df['bill_rank']
    return (cut_df, max_summary, max_bill)
    
def compression(df):
    return np.mean(df.summary_clean.apply(len) / df.bill_clean.apply(len))


In [6]:
if not SINGLE_RECORD:
    SAMPLE, MAX_SUMMARY_LENGTH, MAX_BILL_LENGTH = trim_dataset(DATA_SET, 0, 1)
else:
    SAMPLE = DATA_SET.copy()

Cut 0 records.
Count of records remaining:  455
New min summary length is 2
New max summary length is 28
New min bill length is 54
New max bill length is 128
Compression of summaries to bills is 0.22755538080201299


In [7]:
def split(data, training_size, testing_size, valid_size, shuffle_data=True):
    ''' Takes in a pandas dataframe as data. Returns three BillsDataset objects'''
    assert training_size + testing_size + \
        valid_size == 1, 'Split sizes should sum to 1'

    def split_index(size, index_length):
        '''Converts decimal to # of samples to take'''
        return int(floor(size * index_length))
    # Split into training / testing / validation sets, assign as attributes.
    indices = list(range(len(data)))

    train_split = split_index(training_size, len(indices))
    test_split = split_index(testing_size, len(indices))

    if shuffle_data:
        shuffle(indices)

    training_data = data.iloc[indices[0:train_split]]
    test_data = data.iloc[indices[train_split:train_split + test_split]]
    validate_data = data.iloc[indices[train_split + test_split:]]

    return (BillsDataset(training_data, 'summary_clean', 'bill_clean'),
            BillsDataset(test_data, 'summary_clean', 'bill_clean'),
            BillsDataset(validate_data, 'summary_clean', 'bill_clean'))

In [8]:
class BillsDataset(Dataset):
    """
    Dataset for Congressional Bills
    Adapted from https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    and from https://gist.github.com/kevinzakka/d33bf8d6c7f06a9d8c76d97a7879f5cb.
    """

    def __init__(self, df, summaries_col, bills_col, transform=None):
        self.data = df.reset_index(drop=True)
        self.labels = summaries_col
        self.texts = bills_col
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        summary = self.data.loc[idx, self.labels]
        bill = self.data.loc[idx, self.texts]

        sample = {'summary': summary, 'bill': bill}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [9]:
TRAIN_DATA, TEST_DATA, VALIDATE_DATA = split(SAMPLE, TRAINING_SIZE, TESTING_SIZE, VALIDATION_SIZE, shuffle_data=True)

In [10]:
def build_vocab(training_data, summary_col='summary', bill_col='bill', summaries=True, bills=True):
    '''
    Builds a Vocab object for a Dataset object.
    If default BillsDataset object, summary and bill are dict keys.
    '''
    counter_words = Counter()

    for index in range(len(training_data)):
        example = training_data[index]
        if summaries:
          counter_words.update(example[summary_col])
        if bills:
          counter_words.update(example[bill_col])

    return Vocab(counter_words)

In [11]:
VOCAB = build_vocab(TRAIN_DATA, 'summary','bill')
VOCAB_SIZE = len(VOCAB)
PAD_TOKEN = VOCAB.stoi['<pad>']
print(f'Vocab size is {VOCAB_SIZE}')

Vocab size is 2583


With the VOCAB object, we'll leverage GLoVe pretrained embeddings. GLOVE_VECS is size (VOCAB, 300).

In [12]:
def build_glove(vocab):
  '''
  Return the pretrained embeddings for the vocab words.
  '''
  # https://nlp.stanford.edu/projects/glove/
  VECTORS_CACHE_DIR = './.vector_cache'
  glove = GloVe(name='6B', cache=VECTORS_CACHE_DIR)
  glove_vectors = glove.get_vecs_by_tokens(vocab.itos)
  return glove_vectors

In [13]:
GLOVE_VECS = build_glove(VOCAB)
print(GLOVE_VECS.size())

./.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                               
100%|█████████▉| 399999/400000 [00:58<00:00, 6873.56it/s]


torch.Size([2583, 300])


In [15]:
def get_dataloaders(batch_size, vocab, max_summary_length, max_bill_length, **kwargs):
    '''
    kwargs for training_data:, test_data:, and validation_data:.
    Returns dict of dataloaders based on arg name input
    '''
    dataloaders = {}
    # Set params for the collate function
    collate_fn = partial(
        
        collate_bills_fn, vocab=vocab, max_summary_length=max_summary_length, max_bill_length=max_bill_length)

    for dataset_name, data in kwargs.items():
        if len(data) > 0:
            dataloaders[dataset_name] = DataLoader(
                data, batch_size=batch_size,
                shuffle=True, collate_fn=collate_fn)

    return dataloaders

In [16]:
def collate_bills_fn(batch, vocab, max_summary_length=512, max_bill_length=2048):
    '''
    Collates the batches into the dataloader. Pads unequal lengths with zeros
    based on the max lengths given.
    '''
    labels = []
    texts = []
    for idx, text_dict in enumerate(batch):
        # Get the label and the text
        label = text_dict['summary']
        # Reversing it may improve performance re / research
        text = text_dict['bill'] #[::-1]
        # Output for the sample
        label_vectors = []
        text_vectors = []
        # Check lengths; see how much to pad
        label_length = len(label)
        text_length = len(text)
        labels_to_pad = max_summary_length - label_length
        text_to_pad = max_bill_length - text_length

        if label_length < max_summary_length:
            label.extend(['<pad>'] * labels_to_pad)

        if text_length < max_bill_length:
            text.extend(['<pad>'] * text_to_pad)

        for word in label:
            label_vectors.append(vocab.stoi[word])
        for word in text:
            text_vectors.append(vocab.stoi[word])


        labels.append(torch.LongTensor(label_vectors))
        texts.append(torch.LongTensor(text_vectors))
    # Returns shape of (batch size, max_summary (or bill)_length) for each
    return (torch.stack(labels), torch.stack(texts))

In [30]:
DATALOADERS_DICT = get_dataloaders(BATCH_SIZE, VOCAB, MAX_SUMMARY_LENGTH, MAX_BILL_LENGTH, train_data=TRAIN_DATA, test_data=TEST_DATA, validate_data=VALIDATE_DATA)

This results in (number of samples in each set / batch size) tuples (or iteration steps) of size (batch size, max length).

So if size of training is 15, and batch size is 5, enumerating through dataloader will have 3 steps of inputting label (batch size, summary_length) and text (batch size, bill length).

We can view the data:

In [18]:
print(DATALOADERS_DICT.keys())

l, f = next(iter(DATALOADERS_DICT['train_data']))

if BATCH_SIZE > 1:
    print("Summary:", l, [VOCAB.itos[x] for x in l[0].squeeze(0)], '\n')
    print("Bill:", f, [VOCAB.itos[x] for x in f[0].squeeze(0)])
else:
    print("Summary:", l, '\n')
    print("Bill:", f)

dict_keys(['train_data', 'test_data', 'validate_data'])
Summary: tensor([[   9,  915,  201,    5, 1039,   15,  735, 1054,    5,  827,    2,  715,
            4,  530,    4,   10,   11,    3,    8,    1,    1,    1,    1,    1,
            1,    1,    1,    1],
        [   9,  638,  440,  119, 1567,   11,    4,   14,   64,    2,  145,   11,
           10,   37,  259,    2,   27,   26,    3,  166,  127,   24,  111,  103,
          141,    3,    8,    1],
        [   9, 1019,  440,   63,   93,   11,  152,  274,  204,   15,  160,   15,
          441,    6, 1741,  836,  194,   51,   14,    3,    8,    1,    1,    1,
            1,    1,    1,    1],
        [   9,   10,   37,  434,    2,  330,    4,  162, 2099,    6,  439,  447,
          115,  377,  452,    6, 1057,  946, 1358,  198,   15,  162, 2100,    3,
            8,    1,    1,    1],
        [   9, 1308,    6, 1309, 1179,  184,   20,    6,  386,  822,   11,    4,
           14,   10,   37,  507,    2,  183,    4, 2102, 1838,    8,  

In [19]:
# ![Image](seq2seq.png)
# Image Credit to Yoav Goldberg.

## Model Creation

Our model is a Sequence To Sequence model with an LSTM-based Encoder and Decoder.

Given an input sequence (the text of a Congressional bill) $W_{0:n}$ of length $n$, we produce a target sequence (bill summary), $U_{0:m}$ of length $m$, where $n$ > $m$. Let $<sos>$ and $<eos>$ be tokens indicating the start and end of text, respectively; meaning $W_{0}, U_{0} = <sos>$ and $W_{n}, U_{m} = <eos>$.

Furthermore, we define embedding $E \in n \times e$, hidden dimension $H$, and vocab size $|V|$. Additionally, for LSTM $L$ in both the $ENCODER$ and $DECODER$, the hidden state $h$ and cell state $c$ components of state $s$ are initialized as $h_0, c_0 = \vec{0}$. Lastly, $L_{j}$ is layer $j$ of the LSTM $L$. 

The Sequence To Sequence model is broadly defined as:

$\underset{n \times H}{C} = ENCODER_{RNN}(W_{0:n})$

$\underset{1 \times |V|}{\hat{u}}= DECODER_{RNN}(C; \hat{y}_{0:m-1}; s_{0:m-1})$

where $ENCODER$ produces context vector $C$ from text $W$ as follows: 

At step $i$ of input sequence $W_{0:n}$, for $0 \leq i \leq n$:

$\underset{1 \times e}{x_i} = E_{W_i}$

and passing to $x_{i:n}$ PyTorch, we obtain:

$\underset{n \times H}{\hat{y}_{i:n}}, \underset{1 \times H}{h_1}, \underset{1 \times H}{c_1} = LSTM_{L_{0}}(x_{i} , (h_{0}, c_{0}))$

And for layer $j$ of LSTM $L$,

$\underset{1 \times H}{\hat{y}^{j}_{i:n}}, \underset{1 \times H}{h_1}, \underset{1 \times H}{c_1}= LSTM_{L_{j}}(\hat{y}^{j-1}_{i:n}, (h^{j-1}, c^{j-1}))$

where the last layer produces ${C}$ from the final state $s^{enc}$, made up of the final $h$ and $c$ results.

Next, $DECODER$ decodes context vector $C$ in combination with state $s_{i-1}$ and predicted value $\hat{y}_{i-1}$, producing $\hat{u}_{1:m}$ as follows:

$\underset{1 \times |V|}{\hat{y}_i} = O(LSTM_{L_{i}}(c; \hat{y}_{i-1}; s_{i-1}))$

in which $O$ is the softmax function and $h_0, c_0 = C$. $\hat{y}_i$ is the predicted probability distribution for $w_{i+1}$.

For each $\hat{y}_i$ of the $DECODER$, we calculate the loss $\ell$ by $\log \widehat{y}_{i_{[w_{i+1}]}}$. Across $\hat{y}_{1:m}$, it is the average cross entropy loss, $\ell = \frac{1}{m}\sum_{i=1}^{m} \log \widehat{y}_{i_{[w_{i+1}]}}$.

With this step, we preform end to end backpropagation through the $DECODER$ to the $ENCODER$.


In [31]:
class Encoder(nn.Module):
    def __init__(self,
                 input_size,
                 hidden_size,
                 pretrained_embeddings,
                 num_layers,
                 dropout,
                 pad_token,
                 freeze_glove=False,
                 ):
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding.from_pretrained(
            pretrained_embeddings, padding_idx=pad_token, freeze=freeze_glove)

        self.rnn = nn.LSTM(self.input_size, self.hidden_size, num_layers=self.num_layers, batch_first=False, dropout=dropout)

    def forward(self, text):
      '''
      Text size is batch size, bill length
      '''
      # Embedded size is sequence length, batch size, glove vecs size
      embedded = self.embedding(text) #.view(-1, len(text), 300)
      # This above is batch first, but the rnn takes seq length, batch, size
      embedded = embedded.permute(1, 0, 2)
      # outputs is size (sequence length, batch_size, hidden_size)
      outputs, hidden = self.rnn(embedded)
      # each element in hidden is size (num_layers * num_directions (which is 1 unless using a bidirectional LSTM), batch_size, hidden_size)
      return outputs.float(), tuple([v.float() for v in hidden])

In [32]:
class Decoder(nn.Module):
    def __init__(self, input_size, output_dim, hidden_dim, pretrained_embeddings, num_layers, dropout, pad_token, freeze_glove=False):
        super().__init__()

        self.input_size = input_size
        # output dim should equal VOCAB size
        self.output_dim = output_dim
        # hidden_dim should equal the hidden_dim of the ENCODER
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = nn.LSTM(self.input_size, hidden_dim, num_layers=self.num_layers, batch_first=False, dropout=dropout)

        self.fc_out = nn.Linear(hidden_dim, output_dim) # transfer to device?
        
        self.softmax = nn.Softmax(dim=1)
        self.embedding = nn.Embedding.from_pretrained(
            pretrained_embeddings, padding_idx=pad_token, freeze=freeze_glove)

    def forward(self, input_word, hidden):
 
        embedded = self.embedding(input_word)
        embedded = embedded.permute(1, 0, 2)
        output, hidden= self.rnn(embedded, hidden)
        linear = self.fc_out(output.squeeze(0))
        # prediction = self.softmax(linear)

        return linear, hidden

In [33]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self, label, text):
        batch_size = label.shape[0]
        label_length = label.shape[1]
        vocab_size = self.decoder.output_dim

        decode_outputs = torch.zeros(label_length, batch_size, vocab_size)
        output, hidden0 = self.encoder(text)
        input_word = label[:, 0]

        for t in range(1, label_length):
            input_word = input_word.unsqueeze(1)
            output, hidden = self.decoder(input_word, hidden0)
            decode_outputs[t] = output
            # top_choice = torch.max(output, dim=1)[1]
            input_word = label[:, t]
            hidden0 = hidden

        # return top_choice, decode_outputs
        return decode_outputs

In [92]:
INPUT_SIZE = GLOVE_VECS.size()[1] #300
HIDDEN_SIZE = 512
NUM_LAYERS = 4
GRAD_CLIP = 2
DROPOUT=.5
LOG_INTERVAL = 10

In [93]:
encoder = Encoder(INPUT_SIZE, HIDDEN_SIZE, GLOVE_VECS, NUM_LAYERS, DROPOUT, PAD_TOKEN, False).to(DEVICE)
decoder = Decoder(INPUT_SIZE, VOCAB_SIZE, HIDDEN_SIZE, GLOVE_VECS, NUM_LAYERS, DROPOUT, PAD_TOKEN, False).to(DEVICE)
seq2seq = Seq2Seq(encoder, decoder).to(DEVICE)

## Training

In [94]:
OPTIMIZER = optim.Adam(decoder.parameters(), lr=.0003)
OPTIMIZER2 = optim.Adam(encoder.parameters(), lr=.0003)
LOSS_FN = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN) 

In [95]:
def train_an_epoch(model, dataloader, optimizer, optimizer2, loss_function, log_interval, device):
    model.train()
    total_loss = 0
    for idx, (label, text) in enumerate(dataloader):
        model.zero_grad()
        if USE_CUDA:
          label = label.to(device)
          text = text.to(device)
        outputs = model(label, text)
        output_dim = outputs.shape[-1]
        
        output = outputs.view(-1, output_dim)
        target = label.view(-1)

        if USE_CUDA:
          output = output.to(device)
          target = target.to(device)

        loss = loss_function(output, target)
        loss.backward()
        total_loss += loss.item()

        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
              
        optimizer.step()
        optimizer2.step()


        if idx % log_interval == 0 and idx > 0:
            print(f'Iteration: {idx}; Loss: {loss:.3f}.')
        
    return total_loss / len(dataloader)

In [96]:
def get_score(model, dataloader, loss_function, device):
  model.eval()
  total_loss = 0
  with torch.no_grad():
    for i, (label, text) in enumerate(dataloader):
      if USE_CUDA:
        label = label.to(device)
        text = text.to(device)
      outputs = model(label, text)
      output_dim = outputs.shape[-1]
      
      output = outputs.view(-1, output_dim)
      target = label.view(-1)

      if USE_CUDA:
        output = output.to(device)
        target = target.to(device)

      loss = loss_function(output, target)
      total_loss += loss.item()

    return total_loss / len(dataloader)

In [97]:
# THIS CELL WRITTEN BY PYTORCH TUTORIAL SEQ2SEQ
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# Author: Sean Robertson
import time
import math


def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

In [98]:
EPOCHS = 5000
validation_scores = []
training_scores = []
minimum_score = np.inf
best_model = None
start = time.time()
for epoch in range(1, EPOCHS + 1):
  training_score = train_an_epoch(seq2seq, DATALOADERS_DICT['train_data'], OPTIMIZER, OPTIMIZER2, LOSS_FN, LOG_INTERVAL, DEVICE)
  print('%s (%d %d%%)' % (time_since(start, epoch / EPOCHS),
                                         epoch, epoch / EPOCHS * 100))
  if SINGLE_RECORD:
    validation_score = get_score(seq2seq, DATALOADERS_DICT['train_data'], LOSS_FN, DEVICE)
  else:
    validation_score = get_score(seq2seq, DATALOADERS_DICT['validate_data'], LOSS_FN, DEVICE)
  validation_scores.append(validation_score)
  training_scores.append(training_score)

  if validation_score < minimum_score:
    minimum_score = validation_score
    model_copy = Seq2Seq(encoder, decoder).to(DEVICE)
    model_copy.load_state_dict(seq2seq.state_dict())
    best_model = model_copy

plt.plot(range(1, EPOCHS+1), training_scores, c='r', label='training loss')
plt.plot(range(1, EPOCHS+1), validation_scores, c='b', label='validation loss')
plt.legend()
plt.show()

Iteration: 10; Loss: 7.211.
0m 6s (- 551m 53s) (1 0%)
Iteration: 10; Loss: 5.861.
0m 13s (- 569m 39s) (2 0%)
Iteration: 10; Loss: 5.756.
0m 20s (- 574m 13s) (3 0%)
Iteration: 10; Loss: 5.686.
0m 27s (- 576m 49s) (4 0%)
Iteration: 10; Loss: 5.676.
0m 34s (- 578m 40s) (5 0%)
Iteration: 10; Loss: 5.632.
0m 41s (- 579m 57s) (6 0%)
Iteration: 10; Loss: 5.645.
0m 48s (- 580m 42s) (7 0%)
Iteration: 10; Loss: 5.710.
0m 55s (- 580m 40s) (8 0%)
Iteration: 10; Loss: 5.714.
1m 2s (- 581m 12s) (9 0%)
Iteration: 10; Loss: 5.620.
1m 9s (- 581m 16s) (10 0%)
Iteration: 10; Loss: 5.823.
1m 16s (- 581m 25s) (11 0%)
Iteration: 10; Loss: 5.675.
1m 23s (- 581m 25s) (12 0%)
Iteration: 10; Loss: 5.600.
1m 30s (- 581m 23s) (13 0%)
Iteration: 10; Loss: 5.863.
1m 37s (- 581m 13s) (14 0%)
Iteration: 10; Loss: 5.757.
1m 44s (- 581m 18s) (15 0%)
Iteration: 10; Loss: 5.854.
1m 51s (- 581m 20s) (16 0%)
Iteration: 10; Loss: 5.719.
1m 59s (- 581m 32s) (17 0%)
Iteration: 10; Loss: 5.579.
2m 6s (- 581m 27s) (18 0%)
Itera

KeyboardInterrupt: 

## Generation

In [66]:
def generate(encoder, decoder, text, batch_size, device, method='sample'):
    start_token = torch.LongTensor([VOCAB.stoi['<sos>']], dtype=torch.long).to(device)
    current_token = torch.cat((start_token, torch.zeros(BATCH_SIZE - 1).to(device))).int().unsqueeze(1)

    t = 0
    words = []
    encoder.eval()
    decoder.eval()
    with torch.no_grad():
        
        if USE_CUDA:
          text = text.to(device)

        enc_out, hidden0 = encoder(text)

        while (t < MAX_SUMMARY_LENGTH) and (torch.sum(current_token) != VOCAB.stoi['<eos>']):
            # Depending on if working with a single record or not may need to change this
            word = current_token #.unsqueeze(1)

            if USE_CUDA:
              word = word.to(device)

            d_out, d_hid = decoder(word, hidden0)
            if method=='sample':
                i = 0
                s = np.random.random()
                while s >= 0:
                    i += 1
                    s -= d_out[:, i][0]
                    
                words.append(VOCAB.itos[i])

            else:
                words.append(VOCAB.itos[d_out[0].argmax(0)])

            batch_supplement = torch.zeros(batch_size - 1)

            if USE_CUDA:
              batch_supplement = batch_supplement.to(device)

            current_token = torch.cat((d_out[0].argmax(0).unsqueeze(0), batch_supplement)).int().unsqueeze(1)
            hidden0 = [x.detach() for x in d_hid]
            t+= 1
    return words

In [67]:
generate(encoder, decoder, f, BATCH_SIZE, DEVICE, 'nH')

TypeError: new() received an invalid combination of arguments - got (list, dtype=torch.dtype), but expected one of:
 * (torch.device device)
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, torch.device device)
      didn't match because some of the keywords were incorrect: dtype
 * (object data, torch.device device)
      didn't match because some of the keywords were incorrect: dtype
